<a href="https://colab.research.google.com/github/shreyansh273/Fantasy-Premire-League/blob/main/FPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>